In [83]:
from pyspark.sql import SparkSession
import os
from dotenv import load_dotenv  

In [84]:
# Update path to match where your JAR file is downloaded - make sure it's absolute path
jdbc_jar_path = "/home/riddhi/Documents/Riddhi-Tech/Projects/Python-Projects/CRUD-Flask/library/postgresql-42.7.3.jar"
spark = SparkSession.builder \
        .appName("PostgreSQLIngestion") \
        .config("spark.jars", jdbc_jar_path) \
        .config("spark.driver.extraClassPath", jdbc_jar_path) \
        .getOrCreate()

In [85]:
spark

In [86]:
print(spark.sparkContext._conf.get('spark.jars'))

/home/riddhi/Documents/Riddhi-Tech/Projects/Python-Projects/CRUD-Flask/library/postgresql-42.7.3.jar


In [87]:
# Load environment variables
load_dotenv()

DB_PASSWORD = os.environ.get('DB_PASSWORD')
DB_NAME = os.environ.get('DB_NAME')
DB_USER = os.environ.get('DB_USER')
SCHEMA_NAME = os.environ.get('SCHEMA_NAME')
REGISTRATION_TABLE = os.environ.get('REGISTRATION_TABLE')
print(DB_PASSWORD, DB_NAME, DB_USER, SCHEMA_NAME, REGISTRATION_TABLE)

123abc456 sampledb riddhi_psql users_schema registration


In [ ]:

#This sets up the metadata to ingest your source table.
# JDBC URL and properties for PostgreSQL
jdbc_url = "jdbc:postgresql://localhost:5432/"+ DB_NAME
jdbc_properties = {
    "user": DB_USER,
    "password": DB_PASSWORD,
    "driver": "org.postgresql.Driver"
}

table_name = f"{SCHEMA_NAME}.{REGISTRATION_TABLE}"
print(jdbc_url, jdbc_properties, table_name)

In [ ]:
# Read data from PostgreSQL into a DataFrame
df = spark.read.jdbc( \
url=jdbc_url, \
table=table_name, \
properties=jdbc_properties \
) 

# Show the schema and first few rows of the DataFrame
df.printSchema()
df.show(1)

In [90]:
# Write the DataFrame to Parquet format
base_dir = "/home/riddhi/Documents/Riddhi-Tech/Projects/Python-Projects/CRUD-Flask/"
output_dir = os.path.join(base_dir, "datasets", "parquet", "registration_data")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
df.write.parquet(f"{output_dir}/output_data", mode="overwrite")

In [ ]:
df2 = spark.read.parquet(f"{output_dir}/output_data")
df2.show(8)